<a href="https://colab.research.google.com/github/LordLean/Extracting-Green-Bonds-Use-of-Proceeds/blob/main/QA_FinBert_Finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weights and Biases

In [ ]:
!pip install wandb

import wandb
wandb.login()

In [3]:
%env WANDB_PROJECT=squad_question_answering
%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=all

env: WANDB_PROJECT=squad_question_answering
env: WANDB_LOG_MODEL=true
env: WANDB_WATCH=all


# Load Model 

In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 8.8 MB/s 
     |████████████████████████████████| 6.6 MB 63.4 MB/s 
     |████████████████████████████████| 120 kB 76.9 MB/s 


In [6]:
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
    
model_checkpoint = "yiyanghkust/finbert-pretrain"

# Instance model with an untrained QA head.
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at yiyanghkust/finbert-pretrain were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at yiyanghk

# Load Dataset

In [7]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 6.9 MB/s 
     |████████████████████████████████| 212 kB 77.4 MB/s 
     |████████████████████████████████| 115 kB 64.7 MB/s 
     |████████████████████████████████| 127 kB 80.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [8]:
# Max feature length
max_length = 384
# Overlap length
doc_stride = 128

In [9]:
def preprocess_dataset(samples):
  """
  Function to handle truncation, padding, and store mapping positions.
  """

  samples["question"] = [question.lstrip() for question in samples["question"]]

  # Tokenize our samples with truncation and padding, but keep the overflows using a stride. 
  tokenized_samples = tokenizer(
      samples["question"],
      samples["context"],
      truncation="only_second",
      max_length=max_length,
      stride=doc_stride,
      padding="max_length",
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
  )

  # Mapping for longer sequences which will give multiples features.
  sample_mapping = tokenized_samples.pop("overflow_to_sample_mapping")
  # Mapping from token to char positions in context. 
  offset_mapping = tokenized_samples.pop("offset_mapping")

  tokenized_samples["start_positions"] = []
  tokenized_samples["end_positions"] = []
  for i, offset in enumerate(offset_mapping):

      input_ids = tokenized_samples["input_ids"][i]
      cls_index = input_ids.index(tokenizer.cls_token_id)
      # Context vs question
      sequence_ids = tokenized_samples.sequence_ids(i)

      # Spanned features
      sample_index = sample_mapping[i]
      answers = samples["answers"][sample_index]
      # If no answers, make cls_index the answer.
      if len(answers["answer_start"]) == 0:
          tokenized_samples["start_positions"].append(cls_index)
          tokenized_samples["end_positions"].append(cls_index)
      else:
          start_char = answers["answer_start"][0]
          end_char = start_char + len(answers["text"][0])

          # curr span start token index
          token_start_index = 0
          # curr span end token index
          token_end_index = len(input_ids) - 1
          while sequence_ids[token_start_index] != (1):
              token_start_index += 1
          while sequence_ids[token_end_index] != (1):
              token_end_index -= 1

          if not (offset[token_start_index][0] <= start_char and offset[token_end_index][1] >= end_char):
              tokenized_samples["start_positions"].append(cls_index)
              tokenized_samples["end_positions"].append(cls_index)
          else:
              while token_start_index < len(offset) and offset[token_start_index][0] <= start_char:
                  token_start_index += 1
              tokenized_samples["start_positions"].append(token_start_index - 1)
              while offset[token_end_index][1] >= end_char:
                  token_end_index -= 1
              tokenized_samples["end_positions"].append(token_end_index + 1)

  return tokenized_samples

In [10]:
from datasets import load_dataset
from transformers import DefaultDataCollator

# Load squad dataset
dataset = load_dataset("squad")
# Apply preprocessing function to all sentence pairs in dataset.
tokenized_dataset = dataset.map(preprocess_dataset, batched=True, remove_columns=dataset["train"].column_names)

# Training and evaluation dataset
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["validation"]

# Data collator to batch the processed examples together.
data_collator = DefaultDataCollator()

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/88 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

# Training Stage 

## Training Args

In [11]:
# default to an instance of AdamW and a scheduler given by get_linear_schedule_with_warmup()
batch_size = 8
learning_rate = 1e-4
num_train_epochs = 2
weight_decay = 0.05

In [12]:
import string
from random import choice
chars = string.digits
run_name = ''.join(choice(chars) for _ in range(7))
print(run_name)

1589230


In [13]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_train_epochs,
    weight_decay = weight_decay,
    fp16 = True,
    # load_best_model_at_end = True,
    report_to = "wandb",
    run_name = run_name,
)

## Train

In [14]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Using cuda_amp half precision backend


In [ ]:
trainer.train()

wandb.finish()

In [16]:
from google.colab import files

trainer.save_model(f"{model_name}-finetuned-squad/model")

Saving model checkpoint to finbert-pretrain-finetuned-squad/model
Configuration saved in finbert-pretrain-finetuned-squad/model/config.json
Model weights saved in finbert-pretrain-finetuned-squad/model/pytorch_model.bin
tokenizer config file saved in finbert-pretrain-finetuned-squad/model/tokenizer_config.json
Special tokens file saved in finbert-pretrain-finetuned-squad/model/special_tokens_map.json


In [20]:
print(run_name)

1892258


In [18]:
!zip -r /content/1892258.zip /content/finbert-pretrain-finetuned-squad/model

  adding: content/finbert-pretrain-finetuned-squad/model/ (stored 0%)
  adding: content/finbert-pretrain-finetuned-squad/model/vocab.txt (deflated 50%)
  adding: content/finbert-pretrain-finetuned-squad/model/tokenizer.json (deflated 70%)
  adding: content/finbert-pretrain-finetuned-squad/model/pytorch_model.bin (deflated 7%)
  adding: content/finbert-pretrain-finetuned-squad/model/training_args.bin (deflated 48%)
  adding: content/finbert-pretrain-finetuned-squad/model/special_tokens_map.json (deflated 42%)
  adding: content/finbert-pretrain-finetuned-squad/model/config.json (deflated 47%)
  adding: content/finbert-pretrain-finetuned-squad/model/tokenizer_config.json (deflated 45%)


In [19]:
files.download("{}.zip".format(run_name))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>